In [1]:
import math
import torch
import torch.nn as nn

In [2]:
from embed import FixedEmbedding

In [3]:
class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='h'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        if freq == 't':
            self.minute_embed = Embed(minute_size, d_model)
        self.hour_embed = Embed(hour_size, d_model)
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)

    def forward(self, x):
        x = x.long()

        minute_x = self.minute_embed(x[:, :, 4]) if hasattr(self, 'minute_embed') else 0.
        hour_x = self.hour_embed(x[:, :, 3])
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])

        return hour_x + weekday_x + day_x + month_x + minute_x

In [4]:
# Define the model dimension and the batch size and sequence length for the example
d_model = 64  # Example: embedding size of 64
batch_size = 2
seq_length = 10

# Instantiate the TemporalEmbedding class with 'fixed' embedding type for the example
temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type='fixed', freq='h')

# Example input tensor representing [batch_size, sequence_length, time_features]
# The values should be in the appropriate range for each time feature
minute_range = (0, 4) if hasattr(temporal_embedding, 'minute_embed') else (0, 1)
hour_range = (0, 24)
weekday_range = (0, 7)
day_range = (0, 32)
month_range = (0, 13)

# Adjusted random input considering the valid range for each embedding layer
example_input = torch.stack([
    torch.randint(*month_range, size=(batch_size, seq_length)),
    torch.randint(*day_range, size=(batch_size, seq_length)),
    torch.randint(*weekday_range, size=(batch_size, seq_length)),
    torch.randint(*hour_range, size=(batch_size, seq_length)),
    torch.randint(*minute_range, size=(batch_size, seq_length)) if hasattr(temporal_embedding, 'minute_embed') else torch.zeros(batch_size, seq_length, dtype=torch.int64)
], dim=-1)

# Pass the input through the embedding layer
output = temporal_embedding(example_input)

print(f"Input size: {example_input.size()}")
print(f"Output size: {output.size()}")


Input size: torch.Size([2, 10, 5])
Output size: torch.Size([2, 10, 64])


5 (n_batch, n_seq) tensors are respectively converted to (n_batch, n_seq, d_model).
They are added.

example_input tensor structure:

Batch 1:
    Time Point 1: [Month, Day, Weekday, Hour, (Minute placeholder)]
    Time Point 2: [Month, Day, Weekday, Hour, (Minute placeholder)]
    ...
    Time Point 10: [Month, Day, Weekday, Hour, (Minute placeholder)]

Batch 2:
    Time Point 1: [Month, Day, Weekday, Hour, (Minute placeholder)]
    Time Point 2: [Month, Day, Weekday, Hour, (Minute placeholder)]
    ...
    Time Point 10: [Month, Day, Weekday, Hour, (Minute placeholder)]


In [13]:
print(example_input)

tensor([[[ 0, 18,  2, 16,  0],
         [ 5, 21,  5,  2,  0],
         [ 4, 26,  2, 12,  0],
         [ 0, 14,  1,  2,  0],
         [12, 22,  4, 12,  0],
         [ 4,  6,  3, 19,  0],
         [ 0, 14,  3, 21,  0],
         [ 6, 28,  4,  6,  0],
         [ 1, 24,  4,  8,  0],
         [ 2,  3,  1, 16,  0]],

        [[ 5, 29,  0, 18,  0],
         [ 4, 14,  5, 20,  0],
         [ 4, 22,  1, 12,  0],
         [ 3,  6,  2, 17,  0],
         [ 7, 14,  4,  0,  0],
         [ 7,  6,  5, 12,  0],
         [ 8, 22,  0, 16,  0],
         [ 1, 12,  0,  7,  0],
         [10,  5,  1,  2,  0],
         [ 0, 26,  2, 18,  0]]])
